### Modelling bacteria populations using a logistic model with diffusion
Our goal is to explore the behavior of a population of bacteria in a petri dish when we insert a small quantity of bacteria in the middle of the dish. We model the petri dish by a one-dimensional interval of length $L=1$ and denote by $u(x,t)$ the concentration of bacteria at position $x$ and time $t$. We assume that the bacteria have a growth rate given by the logistic law $au(1-u/K)$, where $a$ is the birth rate of bacteria at low concentrations, and $K$ is the carrying capacity. We also assume that bacteria diffuse with diffusion constant $D>0$. Thus, our PDE model becomes
$$
\begin{align}
u_t & = D u_{xx} + au(1-u/K), \quad 0<x<1 \\
u_x(0,t) & = 0 = u_x(1,t), \quad t>0
\end{align}
$$
reflecting the fact bacteria cannot escape through the boundaries of the petri dish. 

First, we load the necessary Python modules.

In [2]:
%matplotlib widget

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as colors

Next, we define the following functions:
* `discrete_laplacian` provides a centered finite-difference approximation of the second derivative $u_{xx}$;
* `update_solution` uses the forward Euler method to update the solution to the next time step;
* `update_graph` animates the solution.

In [3]:
def discrete_laplacian(u, bdy):
    # periodic boundary conditions:
    L = -2*u
    L += np.roll(u, -1)
    L += np.roll(u, +1)
    if bdy == 'Dirichlet':
        L[0]  = -2*u[0]+u[1]
        L[-1] = -2*u[-1]+u[-2]
    elif bdy == 'Neumann':
        L[0]  = 2*(-u[0]+u[1])
        L[-1] = 2*(-u[-1]+u[-2])
    return L

def update_solution(f, v, dt, pars, Nframes, Nskip):
    n = 0
    u = f
    v[0, :] = f
    d = pars[0]
    a = pars[1]
    K = pars[2]
    while n<Nframes:
        n += 1
        for k in range(Nskip):
            u += (d*discrete_laplacian(u, 'Neumann') + a*u*(1-u/K)) * dt
        v[n, :] = u
        yield u

def update_graph(u, x, line1):
    line1.set_data(x, u)

In the next cell, we define the parameters of our model, set the initial condition, and run the animation of the solution. Run this cell for $D=0.002, 0.001, 0$ to investigate how different diffusion constants affect the solution. 

In [4]:
# model parameters
L = 1      # domain length
D = 0.002  # diffusion coefficient
a = 2      # growth rate
K = 4      # carrying capacity

# numerical parameters
N = 100          # grid size

# set initial condition
x = np.linspace(0, L, N)
f = 1.2*K*(np.heaviside(x-0.6*L, 0) - np.heaviside(x-0.7*L, 0))

# animation parameters
Nsteps = 500
Nskip = 1

# set time step to satisfy stability criterion (cfl = D*Δt/Δx^2 < 0.5)
cfl = 0.4
dx = L/(N-1)
dt = cfl * dx**2 / D

# compute and animate solution
Nframes = int(Nsteps/Nskip)
v = np.zeros((Nframes, N))
pars = (D/dx**2, a, K)

fig = plt.figure()
ax  = plt.axes(xlim=(0,L), ylim=(np.min(np.min(f)*1.1,0), np.max(np.max(f)*1.1,0)))
line1, = ax.plot([], [], linewidth=2, color = "tab:blue")
line2, = ax.plot(x, f, linewidth=2, color = "tab:green")
line3  = ax.axhline(y=K, c='grey', linewidth=0.5)
ax.set_xlabel('x')
ax.set_ylabel('u')
plt.legend((line2, line1, line3), ('initial condition u(x,0)', 'concentration u(x,t)', 'u=K'),
           bbox_to_anchor=(1.05, 1.0), loc='upper center')
ani = animation.FuncAnimation(fig, update_graph,
                              update_solution(f, v, dt, pars, Nframes, Nskip),
                              fargs=(x, line1), interval=5, blit=True, repeat=False)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finally, we visualize the solution in a space-time contour plot.

In [5]:
fig = plt.figure()
extent = [0, L, 0, Nsteps*dt]
colormapoffset = colors.TwoSlopeNorm(vmin=0, vcenter=K)
plt.imshow(v, cmap=cm.GnBu, norm=colormapoffset, extent=extent, aspect='auto', origin='lower')
plt.title('Solution u(x,t)')
plt.xlabel('Space x')
plt.ylabel('Time t')
plt.colorbar()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …